Signup for pinecone pod and create an index with dimentions 1536. I name mine index 'portfolio', so I will be using that name in the codes here after.

Create a text file. A sample file is presented in the github repo. Keep in mind that you should include information that you want your chatbot to search for example, job, skills, etc etc. Also, since we are using texts, it is not a bad idea to use redundant information in the paragraph. Example: words like email, contact detail, getting on touch, etc are semantically closer, so you might use them in the paragraph describing your contact details. Divinding sections of text is also a good idea to keep relavent information close to each other. After that upload the .txt file in colab.

In [1]:
!conda env list

# conda environments:
#
base                     C:\Users\bipna\anaconda3
gpuenv                   C:\Users\bipna\anaconda3\envs\gpuenv
odeuropa              *  C:\Users\bipna\anaconda3\envs\odeuropa



In [19]:
! pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain pinecone-client[grpc]

Import the text and split it using character splitter /n/n such that new paragraphs are its own chunks.

In [2]:
## My file name is template.txt
with open("../template.txt") as f:
    template = f.read()

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=300,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)

In [4]:
docs = text_splitter.create_documents([template])
print('This should be close to the number of paragraph your text has:',len(docs))

Created a chunk of size 407, which is longer than the specified 300
Created a chunk of size 509, which is longer than the specified 300
Created a chunk of size 376, which is longer than the specified 300
Created a chunk of size 717, which is longer than the specified 300
Created a chunk of size 1337, which is longer than the specified 300
Created a chunk of size 319, which is longer than the specified 300


This should be close to the number of paragraph your text has: 10


In [10]:
## Put in API KEYS and environment variables
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ['PINECONE_API_KEY'] = ''
os.environ['PINECONE_INDEX_NAME'] = 'portfolio'

In [6]:
print(docs)

[Document(page_content="General information and Introduction about Bipin\nBipin is a final year Master's Student in Data Science at Eurecom. He is working as an intern at Open Air Interface as a CI developer and ML engineer. He is passionate about leveraging AI/ML for innovative solutions. Bipin likes playing ping pong, boxing, fishing during rain, and hiking. Bipin is looking for a job in the field of Data and Machine Learning."), Document(page_content="Location Address\nBipin Adhikari is originally from Nepal. He lives in Sophia Antipolis, France. Bipin has been in France since 2022. Bipin came here to do his Master's Degree in Data Science."), Document(page_content="Contact Information\nBipin's email address is adhikari@eurecom.fr. His LinkedIn is linkedin.com/in/bipinadhikari101. He is happy to answer any additional inquiries and questions. His CV and resume can be downloaded from the website adhikari.pages.dev. His GitHub page is github.com/bipinadk"), Document(page_content="Educa

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from tqdm.autonotebook import tqdm

embeddings = OpenAIEmbeddings()
index_name = os.environ['PINECONE_INDEX_NAME'] ## Name of your pinecone index

In [17]:
os.environ['PINECONE_INDEX_NAME']

'portfolio'

In [24]:
# pip install pinecone-client[grpc]
from pinecone.grpc import PineconeGRPC as Pinecone
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index = pc.Index(os.environ['PINECONE_INDEX_NAME'])



<generator object GRPCIndex.list at 0x000002CDA1F47060>


In [33]:
#delete all previous ids to keep things clean
index.delete(delete_all=True)

In [34]:
docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

Below is the code if you want to add more text. Keep in mind that there are multiple ways, below code is useful if you want to add each chunk seperately or, use a textsplitter again to create docs.

In [ ]:
# vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
# vectorstore.add_texts(["More text!"])

Test your vector database with similarity search on any query and see if the results are as you expected. If not, some things can be tried like.
- Checking if the text is chunked correctly
- Check if text has relavent information is a way that computers can understand that they are related (meaning things like pronouns can confuse the search)

In [ ]:
query = "What is his phone number?"
result = docsearch.similarity_search(query)
for item in result:
    print(item)


If needed Test your open ai key with embeddings

In [113]:
import requests

def get_embedding(msg):
    OPENAI_API_KEY = ""
    url = "https://api.openai.com/v1/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    data = {
        "input": msg,
        "model": "text-embedding-ada-002"
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise error for bad response status

        responseData = response.json()
        return responseData
    except requests.exceptions.HTTPError as err:
        print(f"Error fetching data: {err}")
        raise  # Re-raise the error to handle it in the calling code

# Example usage:
embedding = get_embedding("What is his phone number")
print(embedding)


{'object': 'list', 'data': [{'object': 'embedding', 'index': 0, 'embedding': [-0.013752735, 0.011258749, 0.009468028, -0.01655277, -0.020980734, 0.029615264, -0.01738627, 0.005889843, -0.025044043, 0.008849416, 0.022204937, 0.013987157, -0.00071466033, 0.007833589, -0.0023539837, 0.006218685, 0.025343582, -0.008875463, 0.027713845, -0.018206745, -0.031021794, 0.0054535586, -0.0013845527, -0.007247535, -0.018154653, 0.0025477072, 0.005043321, 0.010705254, -0.004476802, -0.028469203, 0.019717462, -0.017503481, -0.010868046, -0.023481233, -0.025890566, 0.02465334, -0.011473636, -0.0033112054, 0.019079315, -0.008432667, 0.028052453, 0.007807542, -0.009324771, -0.019157456, -0.014078321, 0.0016816496, -0.014104367, -0.01304947, -0.002643755, 0.019652346, 0.006850321, 0.017138826, 0.013290403, -0.030865513, -0.0043042414, 0.0042749387, -0.007084742, 0.009409423, -0.028026408, 0.010132223, 0.022048656, -0.015159265, -0.0098457085, -0.01038618, -0.004486569, 0.012567604, 0.005349371, 0.0189490

If needed Test your pinecone api key and url

In [ ]:
def getsimilarity(embeddings):
    PineconeKey = ""
    url = "indexurl/query"
    headers = {
        "Api-Key": PineconeKey,
        "Content-Type": "application/json"
    }
    data = {
        "vector": embeddings,
        "includeMetadata": "True",
        'topK': 3,
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise error for bad response status

        responseData = response.json()
        return responseData
    except requests.exceptions.HTTPError as err:
        print(f"Error fetching data: {err}")
        raise  # Re-raise the error to handle it in the calling code

# Example usage:
results = getsimilarity(embedding['data'][0]['embedding'])
print(results)